# import os
import cv2
import numpy as np
from modules.ftu import Tubule
from modules.slide import Slide
from modules.dsa import DSAFolder, DSAItem, DSA

def generateAnno(contoursColors):

    annos = []
    elems = []

    for q in range(len(contoursColors)):
        cnts = contoursColors[q]['cnt']
        color = contoursColors[q]['color']

        for f in range(len(cnts)):
            cnt = cnts[f][0]
            hie = cnts[f][1]
            # print(hie)
            if hie[3]==-1:
                points = []
                for i in cnt:
                    for j in i:
                        x = int(j[0])
                        y = int(j[1]) 
                        points.append([x, y, 0])

                elem = {
                    "type": "polyline",
                    "lineColor": f"rgb({color})", 
                    "lineWidth": 3, 
                    "fillColor": f"rgba({color}, 0.5)",
                    "points": None,
                    "closed": True
                }

                elem["points"] = points

                if len(points)>1:
                    if (f+1)<len(cnts) and cnts[f+1][1][3]==-1: #No Children
                        pass
                    else: # Get Children
                        z = f+1
                        holes = []
                        while True:
                            hole = []
                            try:
                                cntChild = cnts[z][0]
                                hieChild = cnts[z][1]
                            except:
                                break
                            if hieChild[3]==-1:
                                break
                            for i in cntChild:
                                for j in i:
                                    x = int(j[0]) 
                                    y = int(j[1]) 
                                    hole.append([x, y, 0])
                            holes.append(hole)
                            z+=1
                        elem["holes"] = holes
                    elems.append(elem)

    anno = {
        "name": 'tbm_ai', 
        "description": 'tbm_ai',  
        "elements": None                        
    }
    anno["elements"] = elems
    annos.append(anno)

    return annos



def getContours(mask):

    def smoothBinary(mask2d):
        kn = 2
        iterat = 2
        kernel = np.ones((kn, kn), np.uint8) 
        for _ in range(iterat):
            mask2d = cv2.erode(mask2d, kernel, iterations=1) 
            mask2d = cv2.dilate(mask2d, kernel, iterations=1) 
        return mask2d
    
    mask2 = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    maskSmooth = smoothBinary(mask2)

    thresholdArea = 0.0001

    contours, hierarchy = cv2.findContours(maskSmooth, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    print(f"found {len(contours)} contours")

    c_max = max(contours, key = cv2.contourArea)
    area_min = cv2.contourArea(c_max) * thresholdArea
    
    print(len(contours))
    
    cntheis = sorted(zip(contours, hierarchy[0]), key=lambda b: cv2.contourArea(b[0]) , reverse=True)
    
    mxmx = int(1.0 * len(contours))
    
    contours_filtered = []
    
    for qq in range(mxmx):
        if qq%50==0:
            print(f"processing {qq}/{mxmx}")
        contour = cntheis[qq][0]
        hie = cntheis[qq][1]
        contours_filtered.append((contour,hie))

    return contours_filtered

xx = 95663
yy = 41940

mask = np.zeros((yy, xx, 3), dtype='uint8')

mydir = "/home/manojkumargalla/ResUNet/data/pred_512"

files = os.listdir(mydir)

for file in files:
    lst = file.split('[')[-1].split(']')[0].replace("=", "").replace("x", "").replace("w", "").replace("y", "").replace("h", "").split(",")
    # print(lst)
    x, y, w, h = [int(k) for k in lst]
    mask[y:y+h, x:x+w, :] = cv2.imread( f"{mydir}/{file}",3)

contours = getContours(mask)
contoursColors = [{'cnt': contours, 'color': '255, 20, 20'}]

dsafid = {
    '32-2_PAS_1of2':'6691a0ba0519743b266f59c8',
}

slides2 = [
    '32-2_PAS_1of2'
]

annoooos = {}

for name in slides2:
    config = {
        'fid': dsafid[name],
        'username': 'manojkumargalla',
        'password': '3435isbcmon@G',
        'apiUrl': 'https://athena.rc.ufl.edu/api/v1',
        'layerName': 'tbm_geojson',
        'svsBase': "/blue/pinaki.sarder/nlucarelli/kpmp_new/"
    }

    annos = generateAnno(contoursColors)

    annoooos[name] = annos

    dsa = DSA(config)

    fid = dsafid[name]

    annos = annoooos[name]
    dsa.postAnno(fid, annos)

In [ ]:
def generateAnno(contoursColors):

    annos = []
    elems = []

    for q in range(len(contoursColors)):
        cnts = contoursColors[q]['cnt']
        color = contoursColors[q]['color']

        for f in range(len(cnts)):
            cnt = cnts[f][0]
            hie = cnts[f][1]
            print(hie)
            if hie[3]==-1:
                points = []
                for i in cnt:
                    for j in i:
                        x = int(j[0]) + contoursColors[q]['x'] 
                        y = int(j[1]) + contoursColors[q]['y'] 
                        points.append([x, y, 0])

                elem = {
                    "type": "polyline",
                    "lineColor": f"rgb({color})", 
                    "lineWidth": 3, 
                    "fillColor": f"rgba({color}, 0.5)",
                    "points": None,
                    "closed": True
                }

                elem["points"] = points

                if len(points)>1:
                    if (f+1)0:
                slide = Slide(config, svsname)

                slide.extractForeground()

                slide.createBinary(anno)

                patches = slide.extractPairs(config['patchSize'])

                print(len(patches))

                contoursColors = []
                for key in patches:
                    try:
                        patch = patches[key]
                        contours = segmentTBMPair(patch)
                        x, y = key.split('_')
                        contoursColors.append({'x': int(x), 'y': int(y) , 'cnt': contours, 'color': '255, 20, 20'})
                        cv2.imwrite(f"{outdir}/{key}.png", patch)
                    except:
                        pass
                annos = generateAnno(contoursColors)
                dsa.postAnno(fid, annos)
            else:
                logging.warning(f"No annotation found in {fid} __ {svsname}")
        except:
            logging.warning(f"Exeption in {fid} __ {svsname}")
     

In [ ]:
import cv2
import numpy as np
from modules.ftu import Tubule
import os
from modules.slide import Slide
from modules.dsa import DSAFolder, DSAItem, DSA

def generateAnno(contoursColors):

    annos = []
    elems = []

    for q in range(len(contoursColors)):
        cnts = contoursColors[q]['cnt']
        color = contoursColors[q]['color']

        for f in range(len(cnts)):
            cnt = cnts[f][0]
            hie = cnts[f][1]
            print(hie)
            if hie[3]==-1:
                points = []
                for i in cnt:
                    for j in i:
                        x = int(j[0])
                        y = int(j[1]) 
                        points.append([x, y, 0])

                elem = {
                    "type": "polyline",
                    "lineColor": f"rgb({color})", 
                    "lineWidth": 3, 
                    "fillColor": f"rgba({color}, 0.5)",
                    "points": None,
                    "closed": True
                }

                elem["points"] = points

            #     if len(points)>1:
            #         if (f+1) area_min:
            # if area > amin:
            #     msk_ = mask*0
            #     interThickness = 2
            #     cv2.drawContours(msk_, [contour], -1, [255,255,255], interThickness)
            #     if np.sum(msk_*mask)>0:
            #         contours_filtered.append((contour,hie))

    return contours_filtered

xx = 95663
yy = 41940


mask = np.zeros((yy, xx, 3), dtype='uint8')

mydir = "/home/manojkumargalla/ResUNet/data/pred_512_1/32-2_PAS_1of2"

files = os.listdir(mydir)

for file in files:
    # lst = file.split('[')[-1].split(']')[0].replace("=", "").replace("x", "").replace("w", "").replace("y", "").replace("h", "").split(",")
    # print(lst)
    try:
        x = int(file.split('_')[3].strip('x'))
    except:
        print(f'Unable to parse for {file}')
    y = int(filename.split('_')[4].split('.')[0].strip('y'))
    w, h = 512, 512
    mask[y:y+h, x:x+w, :] = cv2.imread( f"{mydir}/{file}",3)

contours = getContours(mask)
contoursColors = [{'cnt': contours, 'color': '255, 20, 20'}]

dsafid = {
    '32-2_PAS_1of2':'6691a0ba0519743b266f59c8'
}

slides2 = [
    '32-2_PAS_1of2'
]

annoooos = {}

for name in slides2:
    config = {
        'fid': dsafid[name],
        'username': 'manojkumargalla',
        'password': '3435isbcmon@G',
        'apiUrl': 'https://athena.rc.ufl.edu/api/v1',
        'layerName': 'tbm_geojson',
        'svsBase': "/blue/pinaki.sarder/nlucarelli/kpmp_new/"
    }

    annos = generateAnno(contoursColors)

    annoooos[name] = annos
    print('DONE GENERATING ANNOTATIONS!!!')

    dsa = DSA(config)

    fid = dsafid[name]

    annos = annoooos[name]
    dsa.postAnno(fid, annos)

In [ ]:

# Example usage
filename = 'pred_img_0_25600x_33792y.png'
x, y = extract_coordinates(filename)
print(f'x = {x}, y = {y}')


In [ ]:
filename = 'pred_img_0_25600x_33792y.png'

In [ ]:
print(filename.split('x_'))

In [ ]:
filename.split('_')[4].split('.')[0].strip('y')

In [ ]:
import os
import cv2
import numpy as np
from modules.ftu import Tubule
from modules.slide import Slide
from modules.dsa import DSAFolder, DSAItem, DSA

def generateAnno(contoursColors):

    annos = []
    elems = []

    for q in range(len(contoursColors)):
        cnts = contoursColors[q]['cnt']
        color = contoursColors[q]['color']

        for f in range(len(cnts)):
            cnt = cnts[f][0]
            hie = cnts[f][1]
            # print(hie)
            if hie[3]==-1:
                points = []
                for i in cnt:
                    for j in i:
                        x = int(j[0])
                        y = int(j[1]) 
                        points.append([x, y, 0])

                elem = {
                    "type": "polyline",
                    "lineColor": f"rgb({color})", 
                    "lineWidth": 3, 
                    "fillColor": f"rgba({color}, 0.5)",
                    "points": None,
                    "closed": True
                }

                elem["points"] = points

                if len(points)>1:
                    if (f+1)<len(cnts) and cnts[f+1][1][3]==-1: #No Children
                        pass
                    else: # Get Children
                        z = f+1
                        holes = []
                        while True:
                            hole = []
                            try:
                                cntChild = cnts[z][0]
                                hieChild = cnts[z][1]
                            except:
                                break
                            if hieChild[3]==-1:
                                break
                            for i in cntChild:
                                for j in i:
                                    x = int(j[0]) 
                                    y = int(j[1]) 
                                    hole.append([x, y, 0])
                            holes.append(hole)
                            z+=1
                        elem["holes"] = holes
                    elems.append(elem)

    anno = {
        "name": 'tbm_ai', 
        "description": 'tbm_ai',  
        "elements": None                        
    }
    anno["elements"] = elems
    annos.append(anno)

    return annos

In [ ]:
import os
import cv2
import numpy as np
from modules.ftu import Tubule
from modules.slide import Slide
from modules.dsa import DSAFolder, DSAItem, DSA

def generateAnno(contoursColors):

    annos = []
    elems = []

    for q in range(len(contoursColors)):
        cnts = contoursColors[q]['cnt']
        color = contoursColors[q]['color']

        for f in range(len(cnts)):
            cnt = cnts[f][0]
            hie = cnts[f][1]
            # print(hie)
            if hie[3]==-1:
                points = []
                for i in cnt:
                    for j in i:
                        x = int(j[0])
                        y = int(j[1]) 
                        points.append([x, y, 0])

                elem = {
                    "type": "polyline",
                    "lineColor": f"rgb({color})", 
                    "lineWidth": 3, 
                    "fillColor": f"rgba({color}, 0.5)",
                    "points": None,
                    "closed": True
                }

                elem["points"] = points

                if len(points)>1:
                    if (f+1)<len(cnts) and cnts[f+1][1][3]==-1: #No Children
                        pass
                    else: # Get Children
                        z = f+1
                        holes = []
                        while True:
                            hole = []
                            try:
                                cntChild = cnts[z][0]
                                hieChild = cnts[z][1]
                            except:
                                break
                            if hieChild[3]==-1:
                                break
                            for i in cntChild:
                                for j in i:
                                    x = int(j[0]) 
                                    y = int(j[1]) 
                                    hole.append([x, y, 0])
                            holes.append(hole)
                            z+=1
                        elem["holes"] = holes
                    elems.append(elem)

    anno = {
        "name": 'tbm_ai', 
        "description": 'tbm_ai',  
        "elements": None                        
    }
    anno["elements"] = elems
    annos.append(anno)

    return annos



def getContours(mask):

    def smoothBinary(mask2d):
        kn = 2
        iterat = 2
        kernel = np.ones((kn, kn), np.uint8) 
        for _ in range(iterat):
            mask2d = cv2.erode(mask2d, kernel, iterations=1) 
            mask2d = cv2.dilate(mask2d, kernel, iterations=1) 
        return mask2d
    
    mask2 = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    maskSmooth = smoothBinary(mask2)

    thresholdArea = 0.0001

    contours, hierarchy = cv2.findContours(maskSmooth, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    print(f"found {len(contours)} contours")

    c_max = max(contours, key = cv2.contourArea)
    area_min = cv2.contourArea(c_max) * thresholdArea
    
    print(len(contours))
    
    cntheis = sorted(zip(contours, hierarchy[0]), key=lambda b: cv2.contourArea(b[0]) , reverse=True)
    
    mxmx = int(1.0 * len(contours))
    
    contours_filtered = []
    
    for qq in range(mxmx):
        if qq%50==0:
            print(f"processing {qq}/{mxmx}")
        contour = cntheis[qq][0]
        hie = cntheis[qq][1]
        contours_filtered.append((contour,hie))

    return contours_filtered

# xx = 95663
# yy = 41940

xx = 129
yy = 61

mask = np.zeros((yy, xx, 3), dtype='uint8')

mydir = "/home/manojkumargalla/ResUNet/data/pred_512_jul15/S-1909-007149_PAS_1of2"

files = os.listdir(mydir)
print('Got the files!')
for file in files:
    # print(file)
    # lst = file.split('[')[-1].split(']')[0].replace("=", "").replace("x", "").replace("w", "").replace("y", "").replace("h", "").split(",")
    # print(lst)
    try:
        x = int(file.split('_')[3].strip('x'))
    except:
        print(f'Unable to parse for {file}')
    y = int(file.split('_')[4].split('.')[0].strip('y'))
    w, h = 512, 512
    # mask[y:y+h, x:x+w, :] = cv2.imread( f"{mydir}/{file}",3)
    img = cv2.imread( f"{mydir}/{file}",3)
    h, w, _ = img.shape
    if y + h <= yy and x + w <= xx:
        mask[y:y+h, x:x+w, :] = img
    else:
        print(f"Skipping tile {file} due to out-of-bounds placement.")
        print(f'y + h is {y+h}, x + w is {x+w}')
#     img = cv2.imread( f"{mydir}/{file}",3)
#     if img is None:
#         continue
    
#     img_resized = cv2.resize(img, (xx, yy))
#     mask = cv2.add(mask, img_resized)
    
print('Getting Contours')
contours = getContours(mask)
contoursColors = [{'cnt': contours, 'color': '255, 20, 20'}]

# dsafid = {
#     'S-1909-007149_PAS_1of2':'66918e4a0519743b266cee1b',
#     'S-1905-018731_PAS_2of2': '66946f630519743b266f75fa',
#     'S-1908-010066_PAS_1of2': '669473920519743b26705428',
#     '18-162_PAS_4of6':'6694760a0519743b2670d8e0',
#     'S-1905-017738_PAS_1of2':'66946ed40519743b266f75ee',
#     'S-2103-004857_PAS_2of2': '66951e5a0519743b267b8fa8',
#     'S-2106-003588_PAS_1of2': '66951e580519743b267b8fa5',
#     'S-1910-000089_PAS_2of2':  '66951fba0519743b267b8fea',
#     'S-2001-005357_PAS_1of2': '66951e5b0519743b267b8fab',
#     'S-1904-007293_PAS_1of2': '669520ca0519743b267cfe0a'
# }
dsafid = {
    'S-1909-007149_PAS_1of2':'66918e4a0519743b266cee1b'
}


# slides2 = [
#     'S-1909-007149_PAS_1of2',
#     'S-1905-018731_PAS_2of2',
#     'S-1908-010066_PAS_1of2',
#     '18-162_PAS_4of6',
#     'S-1905-017738_PAS_1of2',
#     'S-2103-004857_PAS_2of2',
#     'S-2106-003588_PAS_1of2',
#     'S-1910-000089_PAS_2of2',
#     'S-2001-005357_PAS_1of2',
#     'S-1904-007293_PAS_1of2'
# ]
slides2 = [
    'S-1909-007149_PAS_1of2'
]

annoooos = {}

for name in slides2:
    config = {
        'fid': dsafid[name],
        'username': 'manojkumargalla',
        'password': '3435isbcmon@G',
        'apiUrl': 'https://athena.rc.ufl.edu/api/v1',
        'layerName': 'tbm_geojson',
        'svsBase': "/blue/pinaki.sarder/nlucarelli/kpmp_new/"
    }
    print('Generating annos')
    annos = generateAnno(contoursColors)

    annoooos[name] = annos

    dsa = DSA(config)

    fid = dsafid[name]

    annos = annoooos[name]
    dsa.postAnno(fid, annos)

In [ ]:
dsa.postAnno(fid, annos)

In [ ]:
import os
mydir = "/home/manojkumargalla/ResUNet/data/pred_512_jul15/S-1909-007149_PAS_1of2"

files = os.listdir(mydir)
x_s=[]
y_s=[]
for file in files:
    x = int(file.split('_')[3].strip('x'))
    y= int(file.split('_')[4].split('.')[0].strip('y'))
    x_s.append(x)
    y_s.append(y)
print(min(x_s), max(x_s))
print(min(y_s), max(y_s))

In [ ]:
6656/512

In [ ]:
28672/13

In [ ]:
with open('annos_output.txt', 'w') as file:
    file.write(str(annos))

In [ ]:
li=[1,2,3]
a=str(li)
print(a)


In [1]:
import os
import cv2
import numpy as np
from modules.ftu import Tubule
from modules.slide import Slide
from modules.dsa import DSAFolder, DSAItem, DSA

def generateAnno(contoursColors):

    annos = []
    elems = []

    for q in range(len(contoursColors)):
        cnts = contoursColors[q]['cnt']
        color = contoursColors[q]['color']

        for f in range(len(cnts)):
            cnt = cnts[f][0]
            hie = cnts[f][1]
            # print(hie)
            if hie[3]==-1:
                points = []
                for i in cnt:
                    for j in i:
                        x = int(j[0])
                        y = int(j[1]) 
                        points.append([x, y, 0])

                elem = {
                    "type": "polyline",
                    "lineColor": f"rgb({color})", 
                    "lineWidth": 3, 
                    "fillColor": f"rgba({color}, 0.5)",
                    "points": None,
                    "closed": True
                }

                elem["points"] = points

                if len(points)>1:
                    if (f+1)<len(cnts) and cnts[f+1][1][3]==-1: #No Children
                        pass
                    else: # Get Children
                        z = f+1
                        holes = []
                        while True:
                            hole = []
                            try:
                                cntChild = cnts[z][0]
                                hieChild = cnts[z][1]
                            except:
                                break
                            if hieChild[3]==-1:
                                break
                            for i in cntChild:
                                for j in i:
                                    x = int(j[0]) 
                                    y = int(j[1]) 
                                    hole.append([x, y, 0])
                            holes.append(hole)
                            z+=1
                        elem["holes"] = holes
                    elems.append(elem)

    anno = {
        "name": 'tbm_ai_filtered', 
        "description": 'tbm_ai_filtered',  
        "elements": None                        
    }
    anno["elements"] = elems
    annos.append(anno)

    return annos



def getContours(mask):

    def smoothBinary(mask2d):
        kn = 2
        iterat = 2
        kernel = np.ones((kn, kn), np.uint8) 
        for _ in range(iterat):
            mask2d = cv2.erode(mask2d, kernel, iterations=1) 
            mask2d = cv2.dilate(mask2d, kernel, iterations=1) 
        return mask2d
    
    mask2 = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    maskSmooth = smoothBinary(mask2)

    thresholdArea = 0.0001

    contours, hierarchy = cv2.findContours(maskSmooth, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    print(f"found {len(contours)} contours")

    c_max = max(contours, key = cv2.contourArea)
    area_min = cv2.contourArea(c_max) * thresholdArea
    
    print(len(contours))
    
    cntheis = sorted(zip(contours, hierarchy[0]), key=lambda b: cv2.contourArea(b[0]) , reverse=True)
    
    mxmx = int(1.0 * len(contours))
    
    contours_filtered = []
    
    for qq in range(mxmx):
        if qq%50==0:
            print(f"processing {qq}/{mxmx}")
        contour = cntheis[qq][0]
        hie = cntheis[qq][1]
        contours_filtered.append((contour,hie))

    return contours_filtered

xx = 35873
yy = 58561

mask = np.zeros((yy, xx, 3), dtype='uint8')

mydir = "/blue/pinaki.sarder/manojkumargalla/filtered_image_tiles/filtered_image/"

files = os.listdir(mydir)

for file in files:
    # lst = file.split('[')[-1].split(']')[0].replace("=", "").replace("x", "").replace("w", "").replace("y", "").replace("h", "").split(",")
    # print(lst)
    print(file)
    x = int(file.split('_')[0].strip('x'))
    # x, y, w, h = [int(k) for k in lst]
    y = int(file.split('_')[1].split('.')[0].strip('y'))
    w, h = 512, 512
    # mask[y:y+h, x:x+w, :] = cv2.imread( f"{mydir}/{file}",3)
    img = cv2.imread( f"{mydir}/{file}",3)
    h, w, _ = img.shape
    if y + h <= yy and x + w <= xx:
        mask[y:y+h, x:x+w, :] = img
    else:
        print(f"Skipping tile {file} due to out-of-bounds placement.")
        print(f'y + h is {y+h}, x + w is {x+w}')

contours = getContours(mask)
contoursColors = [{'cnt': contours, 'color': '255, 20, 20'}]

dsafid = {
    '32-2_PAS_1of2':'6691a0ba0519743b266f59c8'
}

slides2 = [
    '32-2_PAS_1of2'
]

annoooos = {}

for name in slides2:
    config = {
        'fid': dsafid[name],
        'username': 'manojkumargalla',
        'password': '3435isbcmon@G',
        'apiUrl': 'https://athena.rc.ufl.edu/api/v1',
        'layerName': 'tbm_geojson',
        'svsBase': "/blue/pinaki.sarder/nlucarelli/kpmp_new/"
    }

    annos = generateAnno(contoursColors)

    annoooos[name] = annos

    dsa = DSA(config)

    fid = dsafid[name]

    annos = annoooos[name]
    json_dir = "/home/manojkumaraglla/PostProcess/data/TBM_Annotations"
    annotations_filename = 'tbm_jul18.json'
    with open(f'{json_dir}/{annotations_filename}') as outfile:
        json.dump(annos, outfile)
    # dsa.postAnno(fid, annos)

ModuleNotFoundError: No module named 'modules'

In [ ]:
file='17408x_19456y.png'


In [ ]:
file.split('_')[1]